## **1. Import libraries**

In [1]:
import json
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
from transformers import pipeline


def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(42)

## **2. Load dataset**

In [2]:
math_dataset = "/mnt/VLAI_data/MATH-500/test.jsonl"

with open(math_dataset, "r") as f:
    data = [json.loads(line) for line in f]

## **3. Load Model**

In [3]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


## **4. Inference**

#### Normal prompt

In [4]:
question = data[6]["problem"]

system_prompt = """You will be given a math problem and need to provide a concise factoid answer in Markdown format.
Answer in the form: `Answer: <answer>`. The answer must be a single number or a mathematical expression."""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"Problem: {question}"},
]

outputs = pipe(
    messages,
    max_new_tokens=100,
)
print(f"Problem: {question}")
print(outputs[0]["generated_text"][-1]["content"].strip())

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Problem: What is the smallest positive perfect cube that can be written as the sum of three consecutive integers?
Answer: 1008


#### Chain-of-Thought Prompting

In [5]:
system_prompt_cot = """You will be given a math problem. Think step by step to solve it and showing your intermediate reasoning. 
Answer in the form: `Answer: <single number or expression>`."""

messages = [
    {"role": "system", "content": system_prompt_cot},
    {"role": "user", "content": f"Problem: {question}"},
]
out = pipe(messages, max_new_tokens=2000, do_sample=False)

print(f"Problem: {question}")
print(out[0]["generated_text"][-1]["content"].strip())

/home/binhdt/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/binhdt/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Problem: What is the smallest positive perfect cube that can be written as the sum of three consecutive integers?
To solve this problem, let's consider the general form of three consecutive integers: n, n+1, and n+2.

We want to find the smallest positive perfect cube that can be written as the sum of these three consecutive integers. 

The sum of the three consecutive integers is: n + (n+1) + (n+2) = 3n + 3.

We need to find the smallest positive integer n such that 3n + 3 is a perfect cube.

Let's try to find the smallest perfect cube that can be written in the form 3n + 3.

We can start by listing some perfect cubes: 1, 8, 27, 64, 125,...

We can see that 27 = 3(3) + 3, which is in the form 3n + 3.

However, we need to check if 27 is the smallest perfect cube that can be written in this form.

Let's check the previous perfect cube, which is 8. We can see that 8 = 3(1) + 5, which is not in the form 3n + 3.

Therefore, 27 is indeed the smallest positive perfect cube that can be writte

#### Tree-of-Thought Prompting

In [6]:
prompt_tree = f"""
Imagine three different experts are independently solving this question.
All experts will write down 1 step of their thinking, then share it with the group.
Then all experts will go on to the next step, etc. 
If any expert realises they're wrong at any point then they leave."""


messages = [
    {"role": "system", "content": prompt_tree},
    {"role": "user", "content": f"The question is: {question}"},
]
out = pipe(messages, max_new_tokens=2000, do_sample=False)

print(f"Problem: {question}")
print(out[0]["generated_text"][-1]["content"].strip())

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Problem: What is the smallest positive perfect cube that can be written as the sum of three consecutive integers?
**Expert 1: Math Teacher**

Step 1: To find the smallest positive perfect cube that can be written as the sum of three consecutive integers, let's consider the general form of three consecutive integers: n, n+1, and n+2. The sum of these integers is 3n + 3.

Now, we need to find a perfect cube that can be expressed in the form 3n + 3. Let's start by trying to find the smallest perfect cube that is greater than 3.

Step 2: The smallest perfect cube greater than 3 is 8. We can check if 8 can be expressed in the form 3n + 3. However, 8 cannot be written in this form.

Next, we'll try the next perfect cube, which is 27.

Step 3: We can check if 27 can be expressed in the form 3n + 3. Since 27 = 3(9), we can see that 27 can be written as the sum of three consecutive integers: 9, 10, and 11.

Therefore, the smallest positive perfect cube that can be written as the sum of three co

In [7]:
from collections import Counter
import re

system_prompt_sc = """You will be given a math problem. Think step by step to solve it and showing your intermediate reasoning. 
Answer in the form: `Answer: <single number or expression>`"""


answers = []
samples = 10
for _ in range(samples):
    messages = [
        {"role": "system", "content": system_prompt_sc},
        {"role": "user", "content": f"Problem: {question}"},
    ]
    out = pipe(messages, max_new_tokens=2000, do_sample=True)
    text = out[0]["generated_text"][-1]["content"].strip()
    if "Answer:" in text:
        ans = text.split("Answer:")[-1].strip().split()[0]
    else:
        ans = text.split()[-1]
        ans = re.sub(r"[^0-9]+", "", ans)

    print("=" * 20 + f"Sample {_+1}" + "=" * 20)
    print(f"Answer: {ans}")
    answers.append(ans)

final = Counter(answers).most_common(1)[0][0]
print(f"Problem: {question}")
print(f"Final (self-consistent) Answer: {final}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


====================Sample 1====================
Answer: 27


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


====================Sample 2====================
Answer: 9


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

====================Sample 7====================
Answer: 64


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


====================Sample 8====================
Answer: 93


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


====================Sample 9====================
Answer: 
====================Sample 10====================
Answer: 512
Problem: What is the smallest positive perfect cube that can be written as the sum of three consecutive integers?
Final (self-consistent) Answer: 27
